In [1]:
import torch
import torch.nn as nn
import numpy as np
from data_loader import data_load
from unet_pytorch import build_unet
from ramps import get_current_consistency_weight, update_ema_variables
from glob import glob
import tensorflow as tf

2023-02-15 22:36:43.461566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 22:36:43.758656: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-15 22:36:45.261973: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-15 22:36:45.262101: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [9]:
BANDS = ["blue", "green", "red", "nir", "swir1", "swir2", "ndvi", "nirv"]
KERNEL_SHAPE = [256, 256]
KERNEL_BUFFER = [128, 128]
X_BUFFER, Y_BUFFER = [buffer // 2 for buffer in KERNEL_BUFFER]
BUFFERED_SHAPE = [kernel + buffer for kernel, buffer in zip(KERNEL_SHAPE, KERNEL_BUFFER)]
TARGETS = ["cropland"]
MASK = (slice(None), (X_BUFFER, X_BUFFER + KERNEL_SHAPE[0]), (Y_BUFFER, Y_BUFFER + KERNEL_SHAPE[1]))

def dataset(batch, folder, label=True, buffer_size=6000):
    features = BANDS + TARGETS if label else BANDS
    sub_folder = "labeled" if label else "unlabeled"
    tf_files = glob(f"{folder}/{sub_folder}/*.gz")
    columns = [tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for _feature in features]
    description = dict(zip(features, columns))
    data_func = data_load(tf_files, BANDS, description, response=TARGETS, batch_size=batch, buffer_size=buffer_size)
    data = data_func.get_training_dataset() if label else data_func.get_pridiction_dataset()
    # print(tf_files)
    return data


In [10]:
dataset(32,'/bess23/huaize/semi-supervised/data',label=True)

<BatchDataset element_spec=(TensorSpec(shape=(None, 8, 256, 256), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 256, 256), dtype=tf.float32, name=None))>

In [ ]:
class EpochRunner:
    def __init__(self, steprunner):
        self.steprunner = steprunner
        self.stage = steprunner.stage

    def __call__(self, dataset,test_dataset):
        total_loss, step = 0, 1
        loop = enumerate(dataset.as_numpy_iterator())
        for _, batch in loop:
            features = torch.tensor(batch[0]).to(device)
            labels = torch.tensor(batch[1]).to(torch.int64).to(device)
            loss, step_metrics = self.steprunner(features, labels)
            step_log = dict({self.stage + "_loss": loss}, **step_metrics)
            total_loss += loss
            step += 1
            print(step_log, flush=True)

            loss, step_metrics = self.steprunner(features, labels)
            step_log = dict({self.stage + "_loss": loss}, **step_metrics)
            total_loss += loss
            step += 1
            print(step_log, flush=True)

        epoch_loss = total_loss / step
        epoch_metrics = {
            self.stage + "_" + name: metric_fn.compute().item()
            for name, metric_fn in self.steprunner.metrics_dict.items()
        }
        epoch_log = dict({self.stage + "_loss": epoch_loss}, **epoch_metrics)
        print(epoch_log)

        for name, metric_fn in self.steprunner.metrics_dict.items():
            metric_fn.reset()
        return epoch_log

In [ ]:
train_loader = 
unlabeled_train_loader
NCLASS = 2
model = build_unet(len(BANDS),NCLASS).cuda()
ema_model = build_unet(len(BANDS),NCLASS).cuda()
model = nn.DataParallel(model) 
ema_model = nn.DataParallel(ema_model)
model.to(device)
ema_model.to(device)
